In [1]:
from google.cloud import bigquery
client = bigquery.Client(project='mediar-painel')
import pandas as pd
import pandas_gbq
import numpy as np

In [40]:
query = """SELECT 
'retailer_3_prod' as dataset,
date(transactions.date) as date,
round(sum(total_value),2) as sum_total_value,
count(distinct(transactions.basket_id)) as distinct_basket_id,
count(1) as n_rows,
count(DISTINCT store_id) as n_store
FROM `retailer_3_prod.transaction` as transactions 
JOIN `retailer_3_prod.basket` as bkt
USING (basket_id)
where date(transactions.date) = DATE_SUB(current_date, INTERVAL 3 day)
group by date

UNION ALL

SELECT 
'muffato_storage' as dataset,
dt as date,
round(sum(total_value),2) as sum_total_value,
count(distinct(basket_id)) as distinct_basket_id,
count(1) as n_rows,
count(DISTINCT s_id) as n_store

FROM `muffato_storage.transactions` 
where dt = DATE_SUB(current_date, INTERVAL 3 day)
group by dt

order by date"""

avg = """SELECT
round(avg(sum_total_value),2) as avg_total_value,
round(avg(distinct_basket_id),2) as avg_basket_id,
round(avg(n_rows),2) as avg_n_rows
FROM
(SELECT 
date(transactions.date),
round(sum(total_value),2) as sum_total_value,
count(distinct(transactions.basket_id)) as distinct_basket_id,
count(1) as n_rows 

FROM `retailer_3_prod.transaction` as transactions 
JOIN `retailer_3_prod.basket` as basket 
ON basket.basket_id = transactions.basket_id 
where date(transactions.date) >= DATE_SUB(current_date, INTERVAL 30 day)
group by date(transactions.date))
"""

In [41]:
df = client.query(query).result().to_dataframe()

In [25]:
df1 = df

In [45]:
total_value = df[['dataset','date','sum_total_value']]
basket_id = df[['dataset','date','distinct_basket_id']]
n_rows = df[['dataset','date','n_rows']]
n_store = df[['dataset','date','n_store']]


In [46]:
display(total_value)
display(basket_id)
display(n_rows)
display(n_store)

,dataset,date,sum_total_value
0,retailer_3_prod,2021-03-16,20855576.48
1,muffato_storage,2021-03-16,20855576.48


,dataset,date,distinct_basket_id
0,retailer_3_prod,2021-03-16,168587
1,muffato_storage,2021-03-16,168587


,dataset,date,n_rows
0,retailer_3_prod,2021-03-16,2179408
1,muffato_storage,2021-03-16,2179408


,dataset,date,n_store
0,retailer_3_prod,2021-03-16,67
1,muffato_storage,2021-03-16,67


In [51]:
total_value.loc[-1] = ['diferenças',total_value['date'][0],(total_value['sum_total_value'][0]-total_value['sum_total_value'][1])]

/home/luiz/anaconda3/envs/py38/lib/python3.8/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [52]:
total_value

,dataset,date,sum_total_value
0,retailer_3_prod,2021-03-16,20855576.48
1,muffato_storage,2021-03-16,20855576.48
-1,diferenças,2021-03-16,0.00


In [27]:
df1 = df1.drop([1])

In [28]:
df1.reset_index(drop=True,inplace=True)

In [39]:
df1

,dataset,date,sum_total_value,distinct_basket_id,n_rows
0,muffato_storage,2021-03-16,20855576.48,168587,2179408


In [8]:
df1.groupby('loja').count()

,dataset,date,sum_total_value,distinct_basket_id,n_rows
loja,,,,,
3_11,1,1,1,1,1
3_13,2,2,2,2,2
3_14,2,2,2,2,2
3_15,2,2,2,2,2
3_16,2,2,2,2,2
...,...,...,...,...,...
3_77,2,2,2,2,2
3_78,2,2,2,2,2
3_79,2,2,2,2,2


In [9]:
df1.loc[(df.loja == '3_79'),'distinct_basket_id']=19675
df1

,dataset,loja,date,sum_total_value,distinct_basket_id,n_rows
0,muffato_storage,3_11,2021-03-16,563778.17,4507,68727
1,muffato_storage,3_13,2021-03-16,273078.28,2750,32391
2,retailer_3_prod,3_13,2021-03-16,273078.28,2750,32391
3,muffato_storage,3_14,2021-03-16,139054.55,1899,15530
4,retailer_3_prod,3_14,2021-03-16,139054.55,1899,15530
...,...,...,...,...,...,...
128,retailer_3_prod,3_79,2021-03-16,581023.88,19675,58289
129,muffato_storage,3_8,2021-03-16,319733.93,19675,34579
130,retailer_3_prod,3_8,2021-03-16,319733.93,3052,34579
131,muffato_storage,3_80,2021-03-16,120361.99,1321,14373


In [10]:
for loja in df1['loja'].unique():
    if ((df1.loc[df1['loja'] == loja])['sum_total_value'].diff()).shape[0] != 2:
        print("Não foi processado pelo muffato_production_loader:\n",(df1.loc[df1['loja'] == loja])[['loja','dataset','date']])
        continue
    if ((df1.loc[df1['loja'] == loja])['sum_total_value'].diff()).iloc[1] != 0:
        print ("Processamento incompleto:\n",(df1.loc[df1['loja'] == loja])[['loja','dataset','date','sum_total_value']]) 
    if ((df1.loc[df1['loja'] == loja])['distinct_basket_id'].diff()).iloc[1] != 0:
        print ("Processamento incompleto:\n",(df1.loc[df1['loja'] == loja])[['loja','dataset','date','distinct_basket_id']]) 
    if ((df1.loc[df1['loja'] == loja])['n_rows'].diff()).iloc[1] != 0:
        print ("Processamento incompleto:\n",(df1.loc[df1['loja'] == loja])[['loja','dataset','date','n_rows']])

Não foi processado pelo muffato_production_loader:
    loja          dataset        date
0  3_11  muffato_storage  2021-03-16
Processamento incompleto:
      loja          dataset        date  distinct_basket_id
127  3_79  muffato_storage  2021-03-16                3225
128  3_79  retailer_3_prod  2021-03-16               19675
Processamento incompleto:
     loja          dataset        date  distinct_basket_id
129  3_8  muffato_storage  2021-03-16               19675
130  3_8  retailer_3_prod  2021-03-16                3052


In [11]:
df_avg = pd.read_gbq(query=avg)

In [12]:
df_avg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   loja             67 non-null     object 
 1   avg_total_value  67 non-null     float64
 2   avg_basket_id    67 non-null     float64
 3   avg_n_rows       67 non-null     float64
dtypes: float64(3), object(1)
memory usage: 2.2+ KB


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   dataset             134 non-null    object 
 1   loja                134 non-null    object 
 2   date                134 non-null    object 
 3   sum_total_value     134 non-null    float64
 4   distinct_basket_id  134 non-null    int64  
 5   n_rows              134 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 6.4+ KB


In [18]:
df

,dataset,loja,date,sum_total_value,distinct_basket_id,n_rows
0,muffato_storage,3_11,2021-03-15,402464.91,3190,44543
1,retailer_3_prod,3_11,2021-03-15,402464.91,3190,44543
2,muffato_storage,3_13,2021-03-15,273646.32,2372,28006
3,retailer_3_prod,3_13,2021-03-15,273646.32,2372,28006
4,muffato_storage,3_14,2021-03-15,113022.26,1406,12555
...,...,...,...,...,...,...
129,retailer_3_prod,3_79,2021-03-15,624399.05,2811,64262
130,muffato_storage,3_8,2021-03-15,253245.98,2539,26096
131,retailer_3_prod,3_8,2021-03-15,253245.98,2539,26096
132,muffato_storage,3_80,2021-03-15,96481.73,1087,11019


In [19]:
df_avg

,loja,avg_total_value,avg_basket_id,avg_n_rows
0,3_11,507492.85,4084.30,54093.90
1,3_13,280294.24,2582.58,28207.00
2,3_14,127807.01,1608.56,13199.94
3,3_15,214915.78,2071.95,22134.11
4,3_16,200224.43,2475.67,21541.28
...,...,...,...,...
62,3_77,327459.15,1821.50,30129.56
63,3_78,347955.84,1998.68,33989.63
64,3_79,282566.37,2208.83,27630.44
65,3_8,314528.16,2960.32,30204.05


In [20]:
# avg_num = (df_avg.loc[df_avg['loja'] == '3_11'])['avg_total_value']
# sum_num = (df.loc[df['loja'] == '3_11'])['sum_total_value'][0]
# diff = sum_num - avg_num
# perc = diff/sum_num
# print(diff)
# print(round(perc*100,1))

test = df.set_index('loja').join(df_avg.set_index('loja'))


# for j in df_avg['loja'].unique():
#     if ((df.loc[df['loja'] == j])['sum_total_value'][0] - (df_avg.loc[df_avg['loja'] == j])['avg_total_value'][0]) >= 7:
#         print(j)

In [21]:
test

,dataset,date,sum_total_value,distinct_basket_id,n_rows,avg_total_value,avg_basket_id,avg_n_rows
loja,,,,,,,,
3_11,muffato_storage,2021-03-15,402464.91,3190,44543,507492.85,4084.30,54093.90
3_11,retailer_3_prod,2021-03-15,402464.91,3190,44543,507492.85,4084.30,54093.90
3_13,muffato_storage,2021-03-15,273646.32,2372,28006,280294.24,2582.58,28207.00
3_13,retailer_3_prod,2021-03-15,273646.32,2372,28006,280294.24,2582.58,28207.00
3_14,muffato_storage,2021-03-15,113022.26,1406,12555,127807.01,1608.56,13199.94
...,...,...,...,...,...,...,...,...
3_79,retailer_3_prod,2021-03-15,624399.05,2811,64262,282566.37,2208.83,27630.44
3_8,muffato_storage,2021-03-15,253245.98,2539,26096,314528.16,2960.32,30204.05
3_8,retailer_3_prod,2021-03-15,253245.98,2539,26096,314528.16,2960.32,30204.05


In [22]:
media = pd.DataFrame(data=((test['sum_total_value']-test['avg_total_value'])/test['avg_total_value'])*100,columns=['média']).drop_duplicates()
std = media.std()

In [23]:
media['média'] <= std

ValueError: Can only compare identically-labeled Series objects